In [13]:
import keras
from keras.optimizers import rmsprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras import backend
from keras import optimizers
from keras import backend as K

import scipy.ndimage as im

import numpy as np

import h5py

import tensorflow as tf

import utils
reload(utils)

%matplotlib inline

In [3]:
batch_size = 25

data_root = '../data/tu-berlin/'
train_data_dir = data_root + 'train'
validation_data_dir = data_root + 'test'


def enhance_edges(img):
    k = np.ones((3,3))
    k[1,1] = 4.0
            
    img = np.squeeze(img)    
    factor = 1.0/img.max()
    img = img * factor
    for _ in xrange(3):
        img[img < 0.8] = 0.0
        img = im.convolve(img, k, mode='constant', cval=1.0)
        factor = 1.0/img.max()
        img = img * factor


    img = np.expand_dims(img, axis=2)
    return img

train_datagen = ImageDataGenerator(
        shear_range=0.25,
        zoom_range=0.25,
        rotation_range=35,
        horizontal_flip=True,
        preprocessing_function=enhance_edges)

test_datagen = ImageDataGenerator(preprocessing_function=enhance_edges)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

batch, y = train_generator.next()
img_height = batch.shape[1]
img_width = batch.shape[2]
channels = batch.shape[3]
num_classes = len(y[0])
print img_height, img_width, channels, num_classes

Found 13250 images belonging to 250 classes.
Found 6750 images belonging to 250 classes.
256 256 1 250


In [4]:
print 13250/25, 6750/25

530 270


In [8]:
model = None
load_name = 'sketchnet_filtered2'
with open('../models/'+load_name+'/model.json', 'r') as f:
    model = keras.models.model_from_json(f.read())
    model.load_weights('../models/'+load_name+'/best.h5')

In [9]:
model.pop()
model.add(Dense(num_classes, activation='softmax'))

In [10]:
opt = rmsprop(lr=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer0 (Conv2D)              (None, 256, 256, 64)      640       
_________________________________________________________________
layer1 (Conv2D)              (None, 256, 256, 64)      36928     
_________________________________________________________________
layer2 (MaxPooling2D)        (None, 128, 128, 64)      0         
_________________________________________________________________
layer3 (Conv2D)              (None, 128, 128, 128)     73856     
_________________________________________________________________
layer4 (Conv2D)              (None, 128, 128, 128)     147584    
_________________________________________________________________
layer5 (MaxPooling2D)        (None, 64, 64, 128)       0         
_________________________________________________________________
layer6 (Conv2D)              (None, 64, 64, 256)       295168    
__________

In [11]:
model_name = 'sketchnet_full1'
%mkdir -p ../logs/{model_name}
%mkdir -p ../models/{model_name}

with open('../models/'+model_name+'/model.json', 'w') as f:
    f.write(model.to_json())

log_cb =\
    TensorBoard(log_dir='../logs/'+model_name+'/', 
                histogram_freq=0, 
                write_graph=False, write_images=False)
best_model_cb =\
    ModelCheckpoint('../models/'+model_name+'/best.h5', 
                    monitor='val_acc', verbose=0, 
                    save_best_only=True, 
                    mode='auto', period=1)
latest_model_cb =\
    ModelCheckpoint('../models/'+model_name+'/latest.h5', 
                    monitor='val_acc', verbose=0, 
                    period=1)   

In [12]:
total_epochs = 0

In [16]:
import utils
reload(utils)

trainer =  utils.ModelTrainer(
                 model,
                 'sketchnet_full1',
                 opt,
                 train_generator,
                 validation_generator,
                 train_steps=530,
                 val_steps=270)

In [ ]:
trainer.train(1, 0.00001)

Epoch 1/1
530/530 [==============================] - 356s - loss: 5.6106 - acc: 0.0040 - val_loss: 5.5024 - val_acc: 0.0063


In [ ]:
trainer.train(10, 0.0001)

Epoch 2/11
530/530 [==============================] - 380s - loss: 5.3009 - acc: 0.0274 - val_loss: 4.4963 - val_acc: 0.1148
Epoch 3/11
530/530 [==============================] - 370s - loss: 4.4575 - acc: 0.1081 - val_loss: 3.5027 - val_acc: 0.2656
Epoch 4/11
530/530 [==============================] - 369s - loss: 3.8526 - acc: 0.1751 - val_loss: 3.1188 - val_acc: 0.3048
Epoch 5/11
530/530 [==============================] - 370s - loss: 3.5168 - acc: 0.2342 - val_loss: 2.8204 - val_acc: 0.3681
Epoch 6/11
530/530 [==============================] - 368s - loss: 3.2487 - acc: 0.2717 - val_loss: 2.6284 - val_acc: 0.3919
Epoch 7/11
529/530 [============================>.] - ETA: 0s - loss: 2.8405 - acc: 0.3454

In [ ]:
for layer in model.layers[:-4]:
    layer.trainable = False
    
trainer.train(10, 0.0002)
    

529/530 [============================>.] - ETA: 0s - loss: 2.7010 - acc: 0.3792

In [ ]:
trainer.train(10, 0.0001)

In [20]:
metrics = model.evaluate_generator(validation_generator, steps=135)
print 'val', model.metrics_names, metrics

val ['loss', 'acc'] [2.1733131912019519, 0.51629630834967999]


In [ ]:
trainer.train(50, 0.0001)

Epoch 22/71
530/530 [==============================] - 358s - loss: 2.1148 - acc: 0.4975 - val_loss: 1.9483 - val_acc: 0.5533
Epoch 31/71
530/530 [==============================] - 359s - loss: 2.0188 - acc: 0.5232 - val_loss: 1.9164 - val_acc: 0.5567
Epoch 32/71
530/530 [==============================] - 358s - loss: 1.9567 - acc: 0.5308 - val_loss: 1.8878 - val_acc: 0.5685
Epoch 34/71
530/530 [==============================] - 367s - loss: 2.0601 - acc: 0.5108 - val_loss: 1.9125 - val_acc: 0.5763
Epoch 36/71
529/530 [============================>.] - ETA: 0s - loss: 2.0203 - acc: 0.5248